In [1]:
import pandas as pd
import numpy as np
from statistics import mean, variance

In [2]:
def aggregate_attribute(attr: str, values: list) -> float:
    # number of non-zero values for numbers of calls and sms, mean for everything else
    values = [v for v in values if v == v]
    if attr == 'call' or attr == 'sms':
        return len(values)
    if len(values) == 0:
        return None
    return mean(values)

In [3]:
def create_dataframe(file_path: str) -> object:
    # read csv, split time into separate date and time colums
    data_frame = pd.read_csv(file_path, parse_dates=True)
    data_frame['date'] = pd.to_datetime(data_frame.time.str.split(' ', expand=True)[0])
    data_frame['time'] = data_frame.time.str.split(' ', expand=True)[1]
    return data_frame
    
def get_unique_column_values(column: str, dataframe: object) -> object:
    return dataframe[column].unique()

def get_aggregated_attr_in_window(usr: str, attrib: str, offset: int, dataframe: object) -> list:
    aggregated = []
    # get subtable for user+attribute
    if attr in list(dataframe.groupby(['id']).get_group(usr).variable):
        subframe = dataframe.groupby(['id', 'variable']).get_group((usr, attrib))
    else:
        return
    # using dates from the whole dataframe
    # dates = list(set(subframe.date.tolist()))
    for i in range(len(dates) - offset):
        window_values = []
        for j in range(i, i + offset):
            # selecting attribute values for corresponding dates in window
            window_values.extend(subframe.loc[subframe['date'] == dates[j], 'value'].tolist())
        # aggregate attribute values
        aggregated.append(aggregate_attribute(attrib, window_values))
    return aggregated

def get_targets(usr: str, offset: int, dataframe: object) -> list:
    # get subtable for user+"mood"
    subframe = dataframe.groupby(['id', 'variable']).get_group((usr, "mood"))
    targets = []
    for i in range(len(dates) - offset):
        # calculate target as mean mood value for the day after offset window
        target_mood = aggregate_attribute("mood", subframe.loc[subframe['date'] == dates[i + offset], 'value'].tolist())
        targets.append(target_mood)
    return targets
    
    

In [4]:
dataframe = create_dataframe("dataset_mood_smartphone.csv")
attributes = get_unique_column_values("variable", dataframe)
users = get_unique_column_values("id", dataframe)
window_size = 3
# sort all dates from dataset to iterate over them further
dates = list(set(dataframe.date.tolist()))
dates.sort()
aggregated = {}
targets = {}
for ii, id in enumerate(users):
    # printing because slow
    # print(ii, len(users))
    for attr in attributes:
        aggr = get_aggregated_attr_in_window(id, attr, window_size, dataframe)
        # none check before adding to the resulting dict
        if aggr is not None:
            aggregated[(id, attr)] = aggr
    targets[id] = get_targets(id, window_size, dataframe)

# add target column to aggregated db
attributes = np.append(attributes, 'target')
for id in users:
    aggregated[(id, 'target')] = targets[id]

In [5]:
# verify shape
lens = []
for _, values in aggregated.items():
    lens.append(len(values))
print(variance(lens), 'expected 0')

0 expected 0


In [6]:
# filling table and printing to csv
p = len(next(iter(aggregated.values())))
df_collection = [[None] * (len(attributes) + 1) for _ in range(p*len(users))]

attr_indices = {attr: i for (i,attr) in enumerate(attributes)}
user_indices = {user: i for (i,user) in enumerate(users)}

for key, values in aggregated.items():
    user_index = user_indices[key[0]]
    attr_index = attr_indices[key[1]]
    for i, value in enumerate(values):
        df_collection[user_index * p + i][0] = key[0]
        df_collection[user_index * p + i][attr_index + 1] = value
        
df = pd.DataFrame(df_collection)
df.to_csv("aggregated_3.csv", header=['id']+attributes.tolist(), index=None)